Importing libraries and environment

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

Importing datasets

In [ ]:
data = pd.read_csv("maternal_health_risk_data_set.csv")
data.replace({'high risk': 2, 'mid risk':1, 'low risk': 0}, inplace=True)

Adding additional columns if mother is hypertensive or hypotensive

In [ ]:
def checkBloodPressure(row):
    if row['SystolicBP'] > 180 or row['DiastolicBP'] > 120:
        return 'hb_crisis'
    if row['SystolicBP'] >= 140 or row['DiastolicBP'] >= 90:
        return 'hb_crisis_2'
    if row['SystolicBP'] >= 130 or row['DiastolicBP'] >= 80:
        return 'hb_crisis_1'
    if row['SystolicBP'] >= 120 and row['DiastolicBP'] < 80:
        return 'elevated'
    if row['SystolicBP'] <= 90 or row['DiastolicBP'] <= 60:
        return 'hp'
    return 'normal'
data['BPStatus'] = data.apply (lambda row: checkBloodPressure(row), axis=1)

Encoding the variable with string values

In [ ]:
# label_encoder = LabelEncoder()
# data_Y = np.array(label_encoder.fit_transform(data_Y))
one_hot = OneHotEncoder(handle_unknown='ignore')
data_Y = pd.DataFrame(one_hot.fit_transform(data[['RiskLevel']]).toarray())
data_Y = data_Y.iloc[:].values
tempdata = pd.DataFrame(one_hot.fit_transform(data[['BPStatus']]).toarray())
data = data.join(tempdata)
data_X = data.iloc[:,[True,False,False,True,True,True,False,False,True,True,True,True,True]].values

Splitting the datasets into training, testing and validation and preprocessing

In [ ]:
sc = StandardScaler()
data_X = sc.fit_transform(data_X)
train_X, test_X, train_Y, test_Y = train_test_split(data_X, data_Y,
    test_size=0.2, random_state = 5)

Building the Model

In [ ]:
ann_model = Sequential()
# input layer and 1st hidden layer
ann_model.add(Dense(input_dim = 9, units = 128, activation='relu', kernel_initializer='he_uniform'))
ann_model.add(Dense(units = 64, activation='relu', kernel_initializer='he_uniform'))
ann_model.add(Dense(units = 32, activation='relu', kernel_initializer='he_uniform'))
# output layer
ann_model.add(Dense(units = 3, activation='sigmoid', kernel_initializer='uniform'))
# building the model
ann_model.compile(optimizer=Adam(learning_rate=0.01), loss="binary_crossentropy", metrics=['accuracy'])

Training and Evaluation of the Model

In [ ]:
array_1 = ann_model.get_weights()
np.set_printoptions(threshold=np.inf)
df = pd.DataFrame(array_1)
df.to_csv('initial_weights.csv')
for x in range(5):
    print('Run #', x)
    ann_model.fit(train_X,train_Y,batch_size=512, epochs=1000, verbose=0)
    test_score = ann_model.evaluate(test_X,test_Y, verbose = 0)
    train_score = ann_model.evaluate(train_X,train_Y, verbose = 0)
    predict_Y = ann_model.predict(test_X).round()
    accuracy = accuracy_score(test_Y,predict_Y)
    precision = precision_score(test_Y,predict_Y, average='micro')
    recall = recall_score(test_Y,predict_Y, average='micro')
    f1 = f1_score(test_Y,predict_Y, average='micro')
    print("Loss and Accuracy of Training Dataset", train_score)
    print("Loss and Accuracy of Validation Dataset", test_score)
    print('Accuracy: %.3f' % accuracy)
    print('Precision: %.3f' % precision)
    print('Recall: %.3f' % recall)
    print('F1 Score %.3f' % f1)